## Wilkie et al. 2010
### Rat skeletal muscle

In [1]:
import requests
import openpyxl
import pandas as pd
from time import sleep
from my_utils import get_url, find_duplicate
import my_config

In [8]:
# Excel import
wb = openpyxl.load_workbook('./SourceData/Papers/Wilkie2010/mcp.M110.003129-6.xlsx')
print('Sheet names: ', wb.sheetnames)

# sheet selected
sheet = wb['Extracted']

# convert to df
_df_wilkie = pd.DataFrame(sheet.values)
df_wilkie = pd.DataFrame(_df_wilkie.iloc[3:, 1])
df_wilkie.columns = ['Gene_name']
df_wilkie = df_wilkie.reset_index(drop=True)

Sheet names:  ['Table 1', 'Extracted']


In [10]:
print('Number of genes: ', len(df_wilkie.Gene_name))

Number of genes:  94


In [11]:
# find Uniprot Entry and gene names for each gene
for i in range(len(df_wilkie)):
    # default values
    entry_converted = gene_obtained = 'Not_found'
    
    # gene name
    gene = df_wilkie.iloc[i, 0]

    try:
        # Query the gene name to get the Uniprot ID and the Uniprot-registered gene name
        params = {
        "query": f'gene:{gene} AND organism_id:{my_config.organism_id_list["Homo sapiens"]}',
        "fields": "accession, gene_names",
        "format": "json"
        }
        
        r = get_url(my_config.WEBSITE_API, params=params)
        result = r.json()['results'][0]
        entry_converted = result['primaryAccession']
        gene_obtained = result['genes'][0]['geneName']['value']
    
    except Exception as e:
        print(f'Error in fetching {gene}: {e}')
    
    # Put the obtained Entry and gene name
    df_wilkie.loc[i, 'Entry_Wilkie'] = entry_converted
    df_wilkie.loc[i, 'Gene_name_obtained'] = gene_obtained
    
    # take a break and go next
    sleep(1)
    
    if i % 20 == 0: print(i, entry_converted, gene_obtained)

0 Q9UH99 SUN2
20 Q9BTX1 NDC1
40 Q8NBV4 PLPP7
60 Q9NX61 TMEM161A
80 Q75T13 PGAP1
Error in fetching LOC687797: list index out of range


#### Any duplicates?

In [12]:
print(find_duplicate(df_wilkie.Entry_Wilkie.to_list()))

[]


In [13]:
df_wilkie = df_wilkie.drop(['Gene_name'], axis=1)

In [14]:
df_wilkie.to_csv('./Output/Wilkie_2010.csv', index=False)